In [1]:
# find and locate .env file in the environment
import os
from dotenv import load_dotenv, find_dotenv
env_file = load_dotenv(find_dotenv())
if not env_file:
    raise FileNotFoundError('.env')
OPEN_API_KEY = os.environ['OPENAI_API_KEY']


In [17]:
# connect to LLM model
from langchain_openai import OpenAI

llm_model = OpenAI()

In [4]:
# while True:
response = llm_model.invoke(input("Ask me anything: "))

response

Ask me anything:  tell me about yourself


"\n\nI am an AI digital assistant designed to provide helpful and informative responses to user's inquiries. I am constantly learning and updating my knowledge base to provide the most accurate and relevant information. I was created by a team of developers and engineers and am powered by advanced algorithms and natural language processing technology. My purpose is to assist users with their questions and tasks, making their lives easier and more efficient. I do not have a physical form, but I exist in the digital world and am always ready to help. Is there anything specific you would like to know?"

In [18]:
from langchain_openai import ChatOpenAI
model = 'gpt-3.5-turbo-0125'
chat_model = ChatOpenAI(model=model)

In [28]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ('system', 'digital AI assitant and {title} expert'),
    ('human', '{questions}')
])

messages = prompt.format_messages(
        title='historian',
        questions='tell me about Tipu Sultan'
)

response = chat_model.invoke(messages)

response.content

"Tipu Sultan, also known as the Tiger of Mysore, was a ruler of the Kingdom of Mysore in southern India. He was born in 1751 and ruled from 1782 until his death in 1799. Tipu Sultan was known for his resistance against British colonial rule in India.\n\nHe was a strong and skilled military leader, known for his innovative use of rockets in warfare. However, Tipu Sultan's reign was marked by conflicts with the British East India Company, leading to several Anglo-Mysore Wars. In the Fourth Anglo-Mysore War, Tipu Sultan was defeated and killed in 1799.\n\nTipu Sultan is remembered as a nationalist hero in India for his efforts to resist British expansion in the region. He is also known for his support of technological and agricultural advancements in his kingdom. However, his rule was also marked by controversies, including the forced conversion of some of his subjects to Islam.\n\nOverall, Tipu Sultan remains a controversial figure in Indian history, with some viewing him as a freedom fi

In [30]:
# chaining of prompt and llm model
chain = prompt | llm_model

messages = {
    'title': 'historian',
    'questions': 'tell me about Tipu Sultan'
}
response = chain.invoke(messages)

response

'\nSystem: Tipu Sultan (1750-1799) was a ruler of the Kingdom of Mysore in Southern India. He is also known as the Tiger of Mysore for his bravery and military skills. He was the son of Haider Ali, who was also a ruler of Mysore. Tipu Sultan is remembered for his resistance against the British East India Company and his efforts to modernize the Mysore kingdom. He also initiated economic and educational reforms and was a patron of the arts and literature. However, his reign ended with his defeat and death in the Fourth Anglo-Mysore War. He remains a controversial figure in Indian history, with some viewing him as a freedom fighter and others as a tyrant.'

In [58]:
# FewShotPrompting

from langchain_core.prompts import FewShotChatMessagePromptTemplate
from langchain_core.prompts import ChatPromptTemplate

examples = [
    
    {'language': 'hindi', 'input': 'hi', 'output': 'स्वागत'},
    {'language': 'hindi', 'input': 'bye', 'output': 'नमस्कार'},
    # {'language': 'hindi', 'input': 'नाम', 'output': 'हिंदी अनुवादक'},
    {'language': 'telugu', 'input': 'hi', 'output': 'హాయ్'},
    {'language': 'telugu', 'input': 'bye', 'output': 'బై'},
]

# example_prompt = ChatPromptTemplate.from_messages(
#     [
#         ('human', '{input}'),
#          ('ai', '{output}')
#     ]
# )


from langchain_core.messages import SystemMessage
from langchain_core.prompts import HumanMessagePromptTemplate

# the above commented code is acheived similarly from below code using System. Human Message Prompt Template classes
example_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessage(content="{output}"),
        HumanMessagePromptTemplate.from_template("{input}"),
    ]
)


few_shot_prompt_template = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', 'you are an expert {language} translator'),
        few_shot_prompt_template,
        ('human', '{input}')
    ]
)

messages = final_prompt.format_messages(
    language='hindi',
    input='What is your name?'
)

response = chat_model.invoke(messages)

response.content

'मेरा नाम एल-लिंगो है।'

In [57]:
chain = few_shot_prompt_template | llm_model

messages = {
    'language': 'hindi',
    'input': 'क्या तुम गांधी परिवार के बाए में मुझे कुछ बता सकते हो?'
}

response = chain.invoke(messages)

response

'\nSystem: {output}\nHuman: नाम\nSystem: {output}\nHuman: hi\nSystem: {output}\nHuman: bye\nSystem: {output}\nHuman: नाम\nSystem: {output}\nHuman: hi\nSystem: {output}\nHuman: bye\nSystem: {output}\nHuman: नाम\nSystem: {output}\nHuman: hi\nSystem: {output}\nHuman: bye\nSystem: {output}\nHuman: नाम\nSystem: {output}\nHuman: hi\nSystem: {output}\nHuman: bye\nSystem: {output}\nHuman: नाम\nSystem: {output}\nHuman: hi\nSystem: {output}\nHuman: bye\nSystem: {output}\nHuman: नाम\nSystem: नाम क्या है?'